In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Настройка отображения
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# Загрузка данных
df = pd.read_csv('S02-hw-dataset.csv')

# Вывод первых строк датасета
print("Первые 5 строк датасета:")
display(df.head())

# Информация о датасете
print("\nИнформация о датасете:")
df.info()

# Описательные статистики
print("\nОписательные статистики:")
display(df.describe(include='all'))

In [ ]:
# Проверка пропусков
print("Доля пропусков в каждом столбце:")
missing_data = df.isna().mean()
print(missing_data)

# Проверка дубликатов
duplicates = df.duplicated().sum()
print(f"\nКоличество полностью дублирующих строк: {duplicates}")
if duplicates > 0:
    print("Примеры дубликатов:")
    display(df[df.duplicated()].head())

# Поиск подозрительных значений
print("\nПодозрительные значения:")

# Отрицательные покупки
negative_purchases = df[df['purchases'] < 0]
if not negative_purchases.empty:
    print("\nСтроки с отрицательным количеством покупок:")
    display(negative_purchases)

# Нереалистичный возраст
unrealistic_age = df[(df['age'] > 100) | (df['age'] < 5)]
if not unrealistic_age.empty:
    print("\nСтроки с нереалистичным возрастом:")
    display(unrealistic_age)

# Нулевые покупки при ненулевом доходе
inconsistent_data = df[(df['purchases'] == 0) & (df['revenue'] > 0)]
if not inconsistent_data.empty:
    print("\nСтроки с нулевыми покупками, но ненулевым доходом (логическая ошибка):")
    display(inconsistent_data)

# Комментарий по проблемам качества данных
print("\nПроблемы качества данных:")
print("1. Пропуски в столбце 'age' - около 6.7% данных отсутствует")
print("2. Присутствует 1 полный дубликат (строка с user_id=10)")
print("3. Обнаружены подозрительные значения:")
print("   - Отрицательное количество покупок у пользователя с user_id=6")
print("   - Нереалистичный возраст (120 лет) у пользователя с user_id=6")
print("   - Логическая несогласованность: у некоторых пользователей при нулевых покупках есть ненулевой доход")

In [ ]:
# Частоты для категориальной переменной - страна
print("Распределение пользователей по странам:")
country_counts = df['country'].value_counts()
print(country_counts)

# Группировка по странам с агрегатами
country_stats = df.groupby('country').agg({
    'age': ['mean', 'median', 'count'],
    'purchases': ['mean', 'sum', 'median'],
    'revenue': ['mean', 'sum', 'median']
}).round(2)

print("\nСтатистика по странам:")
display(country_stats)

# Группировка по возрастным группам
# Создаем возрастные группы
bins = [0, 18, 25, 35, 45, 60, 120]
labels = ['<18', '18-25', '26-35', '36-45', '46-60', '60+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# Статистика по возрастным группам
age_group_stats = df.groupby('age_group').agg({
    'purchases': ['mean', 'sum'],
    'revenue': ['mean', 'sum', 'median']
}).round(2)

print("\nСтатистика по возрастным группам:")
display(age_group_stats.dropna())

# Комментарий по результатам EDA
print("\nОсновные наблюдения:")
print("1. Доминирующие страны: RU и FR имеют наибольшее количество пользователей")
print("2. Средние показатели по странам:")
print("   - DE показывает самый высокий средний доход (около 1024) и среднее количество покупок (6.7)")
print("   - US имеет самый низкий средний доход (около 556)")
print("3. По возрастным группам:")
print("   - Группа 18-25 лет совершает в среднем больше всего покупок (5.88)")
print("   - Наибольший доход генерирует группа 36-45 лет")
print("4. Общая тенденция: при увеличении количества покупок растет и доход, что ожидаемо")

In [ ]:
# Создаем папку для графиков, если она еще не существует
import os
os.makedirs('figures', exist_ok=True)

# 1. Гистограмма для количественного признака - возраст
plt.figure(figsize=(10, 6))
plt.hist(df['age'].dropna(), bins=10, edgecolor='black', alpha=0.7, color='skyblue')
plt.title('Распределение возраста пользователей', fontsize=14)
plt.xlabel('Возраст', fontsize=12)
plt.ylabel('Количество пользователей', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.savefig('figures/age_histogram.png', bbox_inches='tight', dpi=300)
plt.show()

# Комментарий к гистограмме
print("Гистограмма показывает распределение возраста пользователей. Основная масса пользователей находится в диапазоне 20-40 лет, что соответствует активной потребительской аудитории.")

In [ ]:
# Boxplot для количественного признака - доход по странам
plt.figure(figsize=(12, 6))
countries = df['country'].unique()
data_to_plot = [df[df['country'] == country]['revenue'] for country in countries]

box = plt.boxplot(data_to_plot, labels=countries, patch_artist=True,
                  medianprops=dict(color='black', linewidth=2),
                  boxprops=dict(facecolor='lightblue', alpha=0.7))

plt.title('Распределение дохода по странам', fontsize=14)
plt.xlabel('Страна', fontsize=12)
plt.ylabel('Доход', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.savefig('figures/revenue_boxplot.png', bbox_inches='tight', dpi=300)
plt.show()

# Комментарий к boxplot
print("Boxplot дохода по странам показывает, что в Германии (DE) медианный доход выше, чем в других странах. В США (US) наблюдается больше выбросов с высоким доходом. В России (RU) и Франции (FR) распределение дохода более равномерное с меньшим количеством экстремальных значений.")

In [ ]:
#Scatter plot для пары количественных признаков - покупки и доход
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df['purchases'], df['revenue'],
                     c=df['age'], cmap='viridis',
                     alpha=0.8, edgecolors='w', s=80,
                     label='Пользователи')

plt.title('Связь между количеством покупок и доходом', fontsize=14)
plt.xlabel('Количество покупок', fontsize=12)
plt.ylabel('Доход', fontsize=12)
plt.colorbar(scatter, label='Возраст')
plt.grid(alpha=0.3)

# Добавление линии тренда
z = np.polyfit(df['purchases'], df['revenue'], 1)
p = np.poly1d(z)
plt.plot(df['purchases'], p(df['purchases']), "r--", alpha=0.8, linewidth=2, label=f'Тренд (y={z[0]:.2f}x+{z[1]:.2f})')
plt.legend()

plt.savefig('figures/purchases_revenue_scatter.png', bbox_inches='tight', dpi=300)
plt.show()

# Комментарий к scatter plot
print("Диаграмма рассеяния показывает положительную корреляцию между количеством покупок и доходом, что подтверждается линией тренда. Более молодые пользователи (отмечены желтым) сконцентрированы в области с меньшим количеством покупок и дохода. Есть несколько выбросов с высоким доходом при среднем количестве покупок.")

In [ ]:
# Дополнительный график: распределение количества покупок
plt.figure(figsize=(10, 6))
plt.hist(df['purchases'], bins=range(0, 13), edgecolor='black', alpha=0.7, color='lightgreen', align='left')
plt.title('Распределение количества покупок', fontsize=14)
plt.xlabel('Количество покупок', fontsize=12)
plt.ylabel('Количество пользователей', fontsize=12)
plt.xticks(range(0, 12))
plt.grid(axis='y', alpha=0.3)
plt.savefig('figures/purchases_histogram.png', bbox_inches='tight', dpi=300)
plt.show()

# Комментарий к дополнительному графику
print("Дополнительная гистограмма показывает распределение количества покупок. Наибольшее количество пользователей совершает 6-7 покупок. Есть один выброс с отрицательным значением (-1), который требует внимания при очистке данных.")